# Pull request scraper

## GitHub credentials
A private access token is necessary to make use of less restrictive API limits.

In [9]:
from github import RateLimitExceededException, Github

# Providing access token
access_token = ""
g = Github(login_or_token=access_token)
-
# Confirm your login is successful
user = g.get_user()
print(f"Authenticated as: {user.login}")

Authenticated as: AbelvdTil


## Files

In [49]:
import os
 
STEP4_TFCOMMITS = os.path.join("data", "step4-tf-commits.json") 
STEP5_TF_PULLREQUESTS = os.path.join("data", "step5-tf-pullrequests.json")
STEP6_TF_REPOS_WITH_PR = os.path.join("data", "step6-tf-repos-with-pr.json")
STEP7_TF_REPOS_COMMITS = os.path.join("data", "step7-tf-repos-commits.json")
STEP8_TF_REPOS_WITH_TF_PR = os.path.join("data", "step8-tf-repos-with-tf-pr.json")
STEP9_TF_KEYWORD_PR = os.path.join("data", "step9-tf-keyword-pr.json")

## File seperator and adder

In [16]:
import json
import math

FILE_TO_SEPERATE = STEP6_TF_REPOS_WITH_PR

nr_parts = 5

### File seperator

Output of step 5 will likely be too large for github, make sure each file is under 100MB

In [17]:
results = open(FILE_TO_SEPERATE)
repoData_dict = json.load(results)

size_per_part = math.ceil(len(repoData_dict) / nr_parts) 

part_data = []

i = 0
current_part = 0
for rp in repoData_dict:
    i += 1
    part = math.floor(i / size_per_part)
    if (part == current_part):
        part_data.append(rp)
    else:
        part_data.append(rp)
        with open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(current_part+1) + ".json", "w") as outfile:
            json.dump(part_data, outfile) 
        part_data = []
        current_part = part
    
if part_data != []:
    with open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(current_part+1) + ".json", "w") as outfile:
            json.dump(part_data, outfile) 

### File combinator

Combine the seperate parts back together

In [10]:
repoData_dict = []
for current_part in range(nr_parts):
    part_file = open(STEP5_TF_PULLREQUESTS.split(".")[0] + "-part-" + str(current_part+1) + ".json", "r")
    part_data = json.load(part_file)

    for rp in part_data:
        repoData_dict.append(rp)

with open(STEP5_TF_PULLREQUESTS, "w") as outfile:
            json.dump(repoData_dict, outfile) 

## STEP 5: Pull request scraping script

Also includes settings, initialization and helper functions.

In [14]:
# SETTINGS
ms_time_between_api_calls = 100
check_limit_every_x_calls = 100
api_calls_per_debug = 1000

# INITIALIZATION

import json
import datetime
import time


# INITIALIZATION
terraform_output = open(STEP4_TFCOMMITS)
step4_output = json.load(terraform_output)

# Retrieve data from previous run
try:
    previous_run = open(STEP5_TF_PULLREQUESTS)
    repoData_dict = json.load(previous_run)
except FileNotFoundError as e:
    repoData_dict = []

iteration = 0
calls_till_next_debug = 0
calls_till_limit_checkup = 0

# Check for api limits, also periodically calls print debug.
def CheckForApiLimit():
    global calls_till_limit_checkup
    global calls_till_next_debug
    global api_calls_per_debug

    # check for limit
    if (calls_till_limit_checkup == 0):
        core_limit = g.get_rate_limit().core

        # sleep when exceeded api core limit
        if (core_limit.remaining <= check_limit_every_x_calls):
            time_to_sleep = core_limit.raw_data['reset'] - time.time() + 1
            print("Rate limit exceeded, sleeping for", time_to_sleep, "seconds.", "Actual remaining calls", core_limit.remaining)
            time.sleep(time_to_sleep)

        calls_till_limit_checkup = check_limit_every_x_calls
    
    calls_till_limit_checkup -= 1

    # check for debug
    if (calls_till_next_debug == 0):
        PrintDebug()
        calls_till_next_debug = api_calls_per_debug

    calls_till_next_debug -= 1

# Prints debug message
def PrintDebug():
    global iteration
    global repo_url

    print(datetime.datetime.now().strftime("%H:%M:%S"), ":", 
              "current iteration:", iteration, 
              "url:", repo_url)

# Pull request scraping script
for rp in step4_output["repositories"]:
    try:
        iteration += 1

        if (iteration > 10):
            break
            
        repo_url = rp["name"]

        # skip already scraped repositories
        if any(d["url"] == repo_url for d in repoData_dict):
            continue

        # Get the repo object from the url
        split_list = repo_url.split("/")
        actual_url = (split_list[3]+ '/' + split_list[4]).split('.git')[0]
        repo = g.get_repo(actual_url)
        
        # Get required info for pull requests
        pull_requests_dict = []
        for pr in repo.get_pulls(state="closed"):

            # retrieve all review comments, not required if there are none.
            comments = []
            if (pr.review_comments > 0):
                for review in pr.get_reviews():
                    if (review.body.strip() != ""):
                        comments.append(review.body)
                CheckForApiLimit()

            # retrieve all connected commits.
            commits = []
            for commit in pr.get_commits():
                commits.append(commit.sha)
            CheckForApiLimit()

            pull_requests_dict.append({"url": pr.html_url, "title": pr.title, "body": pr.body, "comments": comments, "commits": commits})
        
        CheckForApiLimit()        
        repoData_dict.append({"url": repo_url, "pull_requests": pull_requests_dict});
        
        with open(STEP5_TF_PULLREQUESTS, "w") as outfile:
            json.dump(repoData_dict, outfile)
    except Exception as e:
        print("exception:", e)

12:44:46 : current iteration: 1 url: https://github.com/tkhoa2711/terraform-digitalocean.git
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}


## STEP 6: Reduce to repositories with pull requests

In [31]:
step5 = open(STEP5_TF_PULLREQUESTS)
step5_dict = json.load(step5)

output_dict = []
for rp in step5_dict:
    if len(rp["pull_requests"]) != 0:
        output_dict.append({"url": rp["url"], "pull_requests": rp["pull_requests"]})

with open(STEP6_TF_REPOS_WITH_PR, "w") as outfile:
            json.dump(output_dict, outfile) 


## STEP 7: Find tf commits for tf repos with pr's 

This will only store commits that modify a terraform file

In [28]:
from pydriller import Repository

import json

step6 = open(STEP6_TF_REPOS_WITH_PR)
step6_dict = json.load(step6)

terraform_keywords = ['.tf', '.tf.json']

iteration = 0
    
# Pull request scraping script
repo_dic = []
for rp in step6_dict:
    try:
        if (iteration % 50 == 0):
            print("at iteration", iteration)
            with open(STEP7_TF_REPOS_COMMITS, "w") as outfile:
                json.dump(repo_dic, outfile)

        iteration += 1

        repo = Repository(rp["url"])

        # Get each commit
        commit_dic = []
        for commit in repo.traverse_commits():

            modified_terraform = False
            # find if it changes a terraform file
            for file in commit.modified_files:
                if any(key in file.filename for key in terraform_keywords):
                    modified_terraform = True
            
            if modified_terraform:
                commit_dic.append({"hash": commit.hash, 
                                   "url": rp["url"].split(".git")[0] + "/commit/" + commit.hash, 
                                   "date": str(commit.author_date), 
                                   "body": commit.msg})
  
        repo_dic.append({"url":rp["url"], "commits":commit_dic})
        
    except Exception as e:
        print("exception:", e)

with open(STEP7_TF_REPOS_COMMITS, "w") as outfile:
        json.dump(repo_dic, outfile)

at iteration 0
at iteration 50
at iteration 100
at iteration 150
at iteration 200
at iteration 250
at iteration 300
at iteration 350
at iteration 400
at iteration 450
at iteration 500
at iteration 550
at iteration 600


## STEP 8: filter out pull requests without tf commit

Removes any pull request that does not include a commit from the previous step, for the remaining pull requests, it combines the two datasets into one.

In [48]:
from pydriller import Repository

import json

step6 = open(STEP6_TF_REPOS_WITH_PR)
repository_input = json.load(step6)

step7 = open(STEP7_TF_REPOS_COMMITS)
commit_input = json.load(step7)

iteration = 0

output_dict = []

# for each repository
for rp in repository_input:
    # find commits for repo from step 7
    commit_input_list = next(repo["commits"] for repo in commit_input if repo["url"] == rp["url"])

    pr_dict = []
    # for each pull request
    for pr in rp["pull_requests"]:
        commit_dict = []

        # for each commit
        for commit_hash in pr["commits"]:
            # Find the exact commit from step 7
            commit_data = next((commit for commit in commit_input_list if commit["hash"] == commit_hash), None)
            if (commit_data is not None):
                commit_dict.append(commit_data)

        
        pr["commits"] = commit_dict

        if (len(commit_dict) > 0):
            pr_dict.append(pr)
    
    if (len(pr_dict) > 0):
        output_dict.append({"url": rp["url"], "pull_requests": pr_dict})comment

with open(STEP8_TF_REPOS_WITH_TF_PR, "w") as outfile:
    json.dump(output_dict, outfile) 

## STEP 9: list all tf pull request with a keyword

In [63]:
cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step8 = open(STEP8_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step8)

pullrequest_dict_output = []
for repository in repo_input:
    for pr in repository["pull_requests"]:
        
        title   = True if (pr["title"]        is not None and any(key in pr["title"].lower() for key in cost_keywords)) else False
        body    = True if (pr["body"]         is not None and any(key in pr["body"].lower()  for key in cost_keywords)) else False
        comment = True if (any(comment        is not None and key in comment.lower()         for key in cost_keywords for comment in pr["comments"])) else False
        commit  = True if (any(commit["body"] is not None and key in commit["body"].lower()  for key in cost_keywords for commit in pr["commits"])) else False
            
        reason = (("title " if title else "") + 
                  ("body " if body else "") + 
                  ("comment " if comment else "") + 
                  ("commit " if commit else ""))
        
        if (title or body or comment or commit):
            pullrequest_dict_output.append({"reason": reason.strip(), "pull_request": pr})

with open(STEP9_TF_KEYWORD_PR, "w") as outfile:
    json.dump(pullrequest_dict_output, outfile) 

## STEP 10: results

In [118]:
step9 = open(STEP9_TF_KEYWORD_PR)
pr_reason_input = json.load(step9)

print("How often pull requests are in the dataset for each reason: \n")

print("Total PR's found:", len(pr_reason_input))
print("PR's due to keyword in title:", len([pr for pr in pr_reason_input if "title" in pr["reason"]]))
print("PR's due to keyword in description:", len([pr for pr in pr_reason_input if "body" in pr["reason"]]))
print("PR's due to keyword in review comment:", len([pr for pr in pr_reason_input if "comment" in pr["reason"]]))
print("PR's due to keyword in commit message:", len([pr for pr in pr_reason_input if "commit" in pr["reason"]]))

print("\nHow many commits in the dataset not because it's message contains a keyword.\n")

print("Total amount of commits:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"]]))
print("Commits without a keyword:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"] if not any(key in commit["body"].lower() for key in cost_keywords)]))






How often pull requests are in the dataset for each reason: 

Total PR's found: 888
PR's due to keyword in title: 283
PR's due to keyword in description: 457
PR's due to keyword in review comment: 16
PR's due to keyword in commit message: 655

How many commits in the dataset not because it's message contains a keyword.

Total amount of commits: 5000
Commits without a keyword: 4138
